# ECOモデルでの推論

本ファイルでは、ECOモデルを実装し、動画データのクラス分類を行います


# 9.5 学習目標

1.	ECOモデルを実装できる
2.	学習済みのECOモデルを自分のモデルにロードできる
3.	ECOモデルを使用して、テストデータの推論ができる



# 事前準備

- フォルダ「weights」に「ECO_Lite_rgb_model_Kinetics.pth.tar」をダウンロードして配置してください。

https://github.com/mzolfaghari/ECO-pytorch の

https://drive.google.com/open?id=1XNIq7byciKgrn011jLBggd2g79jKX4uD


- 9.2から9.4節での実装内容をフォルダ「utils」に用意しています。

それらを利用します。

In [1]:
import os

import torch
import torch.nn as nn
from torch.nn import init

In [2]:
# フォルダ「weights」が存在しない場合は作成する
weights_dir = "./weights/"
if not os.path.exists(weights_dir):
    os.mkdir(weights_dir)

# フォルダ「weights」に学習済みモデル「ECO_Lite_rgb_model_Kinetics.pth.tar」をダウンロードして配置してください。
# https://github.com/mzolfaghari/ECO-pytorch の
# https://drive.google.com/open?id=1XNIq7byciKgrn011jLBggd2g79jKX4uD

# Kinematics動画データセットのDataLoaderを用意

In [3]:
from utils.kinetics400_eco_dataloader import make_datapath_list, VideoTransform, get_label_id_dictionary, VideoDataset

# vieo_listの作成
root_path = './data/kinetics_videos/'
video_list = make_datapath_list(root_path)

# 前処理の設定
resize, crop_size = 224, 224
mean, std = [104, 117, 123], [1, 1, 1]
video_transform = VideoTransform(resize, crop_size, mean, std)

# ラベル辞書の作成
label_dicitionary_path = './video_download/kinetics_400_label_dicitionary.csv'
label_id_dict, id_label_dict = get_label_id_dictionary(label_dicitionary_path)

# Datasetの作成
# num_segments は 動画を何分割して使用するのかを決める
val_dataset = VideoDataset(video_list, label_id_dict, num_segments=16,
                           phase="val", transform=video_transform, img_tmpl='image_{:05d}.jpg')

# DataLoaderにします
batch_size = 8
val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False)

# 動作確認
batch_iterator = iter(val_dataloader)  # イテレータに変換
imgs_transformeds, labels, label_ids, dir_path = next(
    batch_iterator)  # 1番目の要素を取り出す
print(imgs_transformeds.shape)


torch.Size([8, 16, 3, 224, 224])


# ECOモデルを実装

In [4]:
from utils.eco import ECO_2D, ECO_3D


class ECO_Lite(nn.Module):
    def __init__(self):
        super(ECO_Lite, self).__init__()

        # 2D Netモジュール
        self.eco_2d = ECO_2D()

        # 3D Netモジュール
        self.eco_3d = ECO_3D()

        # クラス分類の全結合層
        self.fc_final = nn.Linear(in_features=512, out_features=400, bias=True)

    def forward(self, x):
        '''
        入力xはtorch.Size([batch_num, num_segments=16, 3, 224, 224]))
        '''

        # 入力xの各次元のサイズを取得する
        bs, ns, c, h, w = x.shape

        # xを(bs*ns, c, h, w)にサイズ変換する
        out = x.view(-1, c, h, w)
        # （注釈）
        # PyTorchのConv2Dは入力のサイズが(batch_num, c, h, w)しか受け付けないため
        # (batch_num, num_segments, c, h, w)は処理できない
        # 今は2次元画像を独立に処理するので、num_segmentsはbatch_numの次元に押し込んでも良いため
        # (batch_num×num_segments, c, h, w)にサイズを変換する

        # 2D Netモジュール 出力torch.Size([batch_num×16, 96, 28, 28])
        out = self.eco_2d(out)

        # 2次元画像をテンソルを3次元用に変換する
        # num_segmentsをbatch_numの次元に押し込んだものを元に戻す
        out = out.view(-1, ns, 96, 28, 28)

        # 3D Netモジュール 出力torch.Size([batch_num, 512])
        out = self.eco_3d(out)

        # クラス分類の全結合層　出力torch.Size([batch_num, class_num=400])
        out = self.fc_final(out)

        return out


In [5]:
net = ECO_Lite()
net

ECO_Lite(
  (eco_2d): ECO_2D(
    (basic_conv): BasicConv(
      (conv1_7x7_s2): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
      (conv1_7x7_s2_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1_relu_7x7): ReLU(inplace)
      (pool1_3x3_s2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
      (conv2_3x3_reduce): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (conv2_3x3_reduce_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2_relu_3x3_reduce): ReLU(inplace)
      (conv2_3x3): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2_3x3_bn): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2_relu_3x3): ReLU(inplace)
      (pool2_3x3_s2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    )
    (inception_a): InceptionA(
      (inception_3a_1x1):

# 学習済みモデルをロード

In [6]:
# 学習済みモデルをロードする関数の定義


def load_pretrained_ECO(model_dict, pretrained_model_dict):
    '''ECOの学習済みモデルをロードする関数
    今回構築したECOは学習済みモデルとレイヤーの順番は同じだが名前が異なる
    '''

    # 現在のネットワークモデルのパラメータ名
    param_names = []  # パラメータの名前を格納していく
    for name, param in model_dict.items():
        param_names.append(name)

    # 現在のネットワークの情報をコピーして新たなstate_dictを作成
    new_state_dict = model_dict.copy()

    # 新たなstate_dictに学習済みの値を代入
    print("学習済みのパラメータをロードします")
    for index, (key_name, value) in enumerate(pretrained_model_dict.items()):
        name = param_names[index]  # 現在のネットワークでのパラメータ名を取得
        new_state_dict[name] = value  # 値を入れる

        # 何から何にロードされたのかを表示
        print(str(key_name)+"→"+str(name))

    return new_state_dict


# 学習済みモデルをロード
net_model_ECO = "./weights/ECO_Lite_rgb_model_Kinetics.pth.tar"
pretrained_model = torch.load(net_model_ECO, map_location='cpu')
pretrained_model_dict = pretrained_model['state_dict']
# （注釈）
# pthがtarで圧縮されているのは、state_dict以外の情報も一緒に保存されているため。
# そのため読み込むときは辞書型変数になっているので['state_dict']で指定する。

# 現在のモデルの変数名などを取得
model_dict = net.state_dict()

# 学習済みモデルのstate_dictを取得
new_state_dict = load_pretrained_ECO(model_dict, pretrained_model_dict)

# 学習済みモデルのパラメータを代入
net.eval()  # ECOネットワークを推論モードに
net.load_state_dict(new_state_dict)


学習済みのパラメータをロードします
module.base_model.conv1_7x7_s2.weight→eco_2d.basic_conv.conv1_7x7_s2.weight
module.base_model.conv1_7x7_s2.bias→eco_2d.basic_conv.conv1_7x7_s2.bias
module.base_model.conv1_7x7_s2_bn.weight→eco_2d.basic_conv.conv1_7x7_s2_bn.weight
module.base_model.conv1_7x7_s2_bn.bias→eco_2d.basic_conv.conv1_7x7_s2_bn.bias
module.base_model.conv1_7x7_s2_bn.running_mean→eco_2d.basic_conv.conv1_7x7_s2_bn.running_mean
module.base_model.conv1_7x7_s2_bn.running_var→eco_2d.basic_conv.conv1_7x7_s2_bn.running_var
module.base_model.conv1_7x7_s2_bn.num_batches_tracked→eco_2d.basic_conv.conv1_7x7_s2_bn.num_batches_tracked
module.base_model.conv2_3x3_reduce.weight→eco_2d.basic_conv.conv2_3x3_reduce.weight
module.base_model.conv2_3x3_reduce.bias→eco_2d.basic_conv.conv2_3x3_reduce.bias
module.base_model.conv2_3x3_reduce_bn.weight→eco_2d.basic_conv.conv2_3x3_reduce_bn.weight
module.base_model.conv2_3x3_reduce_bn.bias→eco_2d.basic_conv.conv2_3x3_reduce_bn.bias
module.base_model.conv2_3x3_reduce_bn.ru

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

# 推論（動画データのクラス分類）

In [7]:
# 推論します
net.eval()  # ECOネットワークを推論モードに

batch_iterator = iter(val_dataloader)  # イテレータに変換
imgs_transformeds, labels, label_ids, dir_path = next(
    batch_iterator)  # 1番目の要素を取り出す

with torch.set_grad_enabled(False):
    outputs = net(imgs_transformeds)  # ECOで推論

print(outputs.shape)  # 出力のサイズ


torch.Size([8, 400])


In [8]:
# 予測結果の上位5つを表示します
def show_eco_inference_result(dir_path, outputs_input, id_label_dict, idx=0):
    '''ミニバッチの各データに対して、推論結果の上位を出力する関数を定義'''
    print("ファイル：", dir_path[idx])  # ファイル名

    outputs = outputs_input.clone()  # コピーを作成

    for i in range(5):
        '''1位から5位までを表示'''
        output = outputs[idx]
        _, pred = torch.max(output, dim=0)  # 確率最大値のラベルを予測
        class_idx = int(pred.numpy())  # クラスIDを出力
        print("予測第{}位：{}".format(i+1, id_label_dict[class_idx]))
        outputs[idx][class_idx] = -1000  # 最大値だったものを消す（小さくする）


# 予測を実施
idx = 0
show_eco_inference_result(dir_path, outputs, id_label_dict, idx)


ファイル： ./data/kinetics_videos/arm wrestling/C4lCVBZ3ux0_000028_000038
予測第1位：arm wrestling
予測第2位：headbutting
予測第3位：stretching leg
予測第4位：shaking hands
予測第5位：tai chi


In [9]:
# 予測を実施
idx = 4
show_eco_inference_result(dir_path, outputs, id_label_dict, idx)


ファイル： ./data/kinetics_videos/bungee jumping/TUvSX0pYu4o_000002_000012
予測第1位：bungee jumping
予測第2位：trapezing
予測第3位：abseiling
予測第4位：swinging on something
予測第5位：climbing a rope


以上